In [1]:
!pip install pymysql

In [3]:
! pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 2.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 2.8 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   ---------------------------------------- 2/2 [sqlalchemy]



In [5]:
# Install cryptography which PyMySQL needs for sha256/caching_sha2 authentication
# Run this cell, then restart the kernel and re-run the connection/upload cells.
!pip install cryptography --quiet
print('Requested installation of cryptography; please restart the kernel and re-run the SQL cell.')


Requested installation of cryptography; please restart the kernel and re-run the SQL cell.


In [11]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

# -----------------------------
# Step 1: Connection settings
# -----------------------------
user = 'root'
password = '1994'
host = '127.0.0.1'
database = 'ola_ride_analysis'
table_name = 'ola_rides'
csv_file = r'D:\Labmentix\Project_7\ola_rides_SQL_PowerBI.csv'

# -----------------------------
# Step 2: Connect to MySQL server (without DB) to ensure DB exists
# -----------------------------
conn = pymysql.connect(host=host, user=user, password=password)
cursor = conn.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
conn.commit()
conn.close()

# -----------------------------
# Step 3: Create SQLAlchemy engine for the database
# -----------------------------
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# -----------------------------
# Step 4: Create table if not exists
# -----------------------------
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Date DATE,
    Time TIME,
    Booking_ID VARCHAR(50) PRIMARY KEY,
    Customer_ID VARCHAR(50),
    Ride_Status VARCHAR(30),
    Booking_Status VARCHAR(30),
    Vehicle_Type VARCHAR(50),
    Ride_Distance INT NULL,
    Booking_Value INT,
    Payment_Method VARCHAR(30),
    Driver_Ratings DECIMAL(3,2) NULL,
    Customer_Rating DECIMAL(3,2) NULL,
    Customer_Cancelled TINYINT,
    Driver_Cancelled TINYINT,
    Cancellation_Reason VARCHAR(100),
    Incomplete_Rides_Flag TINYINT,
    Incomplete_Rides_Reason VARCHAR(100),
    Date_Time DATETIME,
    Hour INT,
    Day INT,
    Is_Weekend TINYINT,
    Month VARCHAR(10)
);
"""

with engine.begin() as conn:  # automatically handles transactions
    conn.execute(text(create_table_query))

# -----------------------------
# Step 5: Load CSV and insert in batches
# -----------------------------
df = pd.read_csv(csv_file)


# Insert data in batches
df.to_sql(
    name=table_name,
    con=engine,
    if_exists='append',  # use 'replace' if you want to overwrite
    index=False,
    chunksize=10000,     # batches of 10k rows
    method='multi'       # faster insertion
)

print("Data inserted successfully!")

Data inserted successfully!
